# 데이터 가공 및 준비

In [1]:
from datasets import load_dataset, load_from_disk, DatasetDict
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

In [2]:
neospectra_dir_path = '/Jupyter/dev_src/fiber-classification/local_data/NeoSpectra'
sagitto_dir_path = '/Jupyter/dev_src/fiber-classification/local_data/Sagitto'

In [3]:
label_idx_dic = {'CO':0, 'LI':1, 'VR':2}
idx_label_dic = {0:'CO', 1:'LI', 2:'VR'}

In [168]:
def load_raw_data(data_dir_path, col_len=258, is_saggitto=False, use_first_sample_only=False):
    # 폴더 이름과 레이블로 사용할 리스트
    labels = ['CO', 'LI', 'VR']

    # 빈 리스트를 만들어서 데이터프레임을 저장할 준비
    all_data = []

    # 각 폴더를 순회
    for label in labels:
        folder_path = os.path.join(data_dir_path, label)

        # 각 폴더 내의 CSV 파일 순회
        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(folder_path, filename)

                # CSV 파일 읽기
                df = pd.read_csv(file_path, index_col=0)
                if is_saggitto:
                    df = df.iloc[:,0:col_len-1]

                # 레이블 및 태그 컬럼 추가
                df['Label'] = label_idx_dic[label]
            
                # 리스트에 데이터프레임 추가
                all_data.append(df[:1] if use_first_sample_only else df)
                # print(f'{filename} {df.shape}')

    # 모든 데이터를 하나의 데이터프레임으로 결합
    final_df = pd.concat(all_data, ignore_index=False)
    # final_df['Label'] = final_df['Label'].astype('category')
    final_df.index.name = 'Index'
    print(f"raw_data's shape: {final_df.shape}")
    
    return final_df

# NeoSpectra 단독 훈련-평가

원본: 높은 파장 -> 낮은 파장 순
- diff 미사용: (0.946031746031746, 0.02944005871585939)
- diff 사용: (0.9936507936507937, 0.007776157913597417)
- 기울기 사용: (0.9936507936507937, 0.012698412698412698)

flip: 낮은 파장 -> 높은 파장 순
- diff 미사용: (0.9619047619047618, 0.02944005871585938)
- diff 미사용: (0.9873015873015873, 0.01851095839633431)
- 기울기 사용: (0.9904761904761905, 0.012698412698412704)

In [169]:
col_len_neospectra = 258

In [231]:
df = load_raw_data(neospectra_dir_path, col_len=col_len_neospectra, use_first_sample_only=False)
df['x'] = df.iloc[:, 0:col_len_neospectra-1].values.tolist()
X = np.array(df['x'].tolist())
# X = np.flip(X, axis=1)
X_diff = np.diff(X, axis=1)
X_relative_gradient = (X[:, 1:] - X[:, :-1]) / X[:, :-1]
X_augmented = np.hstack((X, X_relative_gradient))
y = df['Label'].values

raw_data's shape: (315, 258)


In [232]:
X.shape, X_augmented.shape

((315, 257), (315, 513))

In [233]:
stats = []
for i in range(10):
    # StratifiedKFold 설정
    skf = StratifiedKFold(n_splits=5, shuffle=True)

    # 성능을 저장할 리스트
    accuracies = []

    X = X_augmented

    # 교차 검증 실행
    for train_index, test_index in skf.split(X, y):
        # 훈련 및 테스트 데이터 나누기
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # 표준 정규화 (각 분할마다 train 데이터에 맞춰 scaler fit)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # XGBoost 모델 정의
        model = xgb.XGBClassifier(eval_metric='mlogloss')
        
        # 모델 학습
        model.fit(X_train, y_train)
        
        # 예측
        y_pred = model.predict(X_test)
        
        # 정확도 측정
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        # 성능 출력
        # print(f"Fold Accuracy: {accuracy:.4f}")
        # print(classification_report(y_test, y_pred))

    # 최종 평균 정확도 출력
    # print(f"Average Accuracy: {np.mean(accuracies):.4f}")
    stats.append(np.mean(accuracies))
print(f"Average Accuracy: {np.mean(stats):.4f}, std: {np.std(stats):.4f}")

Average Accuracy: 0.9876, std: 0.0030


-----

# Sagitto 단독 훈련-평가

원본: 낮은 파장 -> 높은 파장 순
- diff 미사용: (0.9540983606557377, 0.02820434513784467)
- diff 사용: (0.9573770491803278, 0.032124455643058075)
- 기울기 사용: (0.9508196721311475, 0.02073624695192377)

In [108]:
col_len_sagitto = 229

In [213]:
df = load_raw_data(sagitto_dir_path, col_len=col_len_sagitto, is_saggitto=True, use_first_sample_only=False)
df['x'] = df.iloc[:, 0:col_len_sagitto-1].values.tolist()
X = np.array(df['x'].tolist())
# X = np.flip(X, axis=1)
X_diff = np.diff(X, axis=1)
X_relative_gradient = (X[:, 1:] - X[:, :-1]) / X[:, :-1]
X_augmented = np.hstack((X, X_relative_gradient))
y = df['Label'].values

raw_data's shape: (305, 229)


In [214]:
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

In [215]:
stats = []
for i in range(10):
    # StratifiedKFold 설정
    skf = StratifiedKFold(n_splits=5, shuffle=True)

    # 성능을 저장할 리스트
    accuracies = []

    X = X_augmented

    # 교차 검증 실행
    for train_index, test_index in skf.split(X, y):
        # 훈련 및 테스트 데이터 나누기
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # 표준 정규화 (각 분할마다 train 데이터에 맞춰 scaler fit)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # XGBoost 모델 정의
        model = xgb.XGBClassifier(eval_metric='mlogloss')
        
        # 모델 학습
        model.fit(X_train, y_train)
        
        # 예측
        y_pred = model.predict(X_test)
        
        # 정확도 측정
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        # 성능 출력
        # print(f"Fold Accuracy: {accuracy:.4f}")
        # print(classification_report(y_test, y_pred))

    # 최종 평균 정확도 출력
    # print(f"Average Accuracy: {np.mean(accuracies):.4f}")
    stats.append(np.mean(accuracies))
print(f"Average Accuracy: {np.mean(stats):.4f}, std: {np.std(stats):.4f}")

Average Accuracy: 0.9511, std: 0.0066


# 결합 훈련 평가: 보류 (*)

왜?
- NeoSpectra와 Sagitto의 각 폴더의 csv 에 있는 인덱스가 동일한 것인지 확인 필요
- Sagitto의 Li 폴더의 일부 CSV 에는 인덱스가 CO로 시작하는 것이 있는데 확인 필요 (예: Sagitto/LI/LI_AC_Y.csv)

결합시: 296건씩 일치

In [ ]:
df_neospectra = load_raw_data(neospectra_dir_path, col_len=col_len_neospectra)
df_neospectra['x'] = df_neospectra.iloc[:, 0:col_len_neospectra-1].values.tolist()
# X = np.array(df['x'].tolist())
# y = df['Label'].values

In [ ]:
df_saggito = load_raw_data(sagitto_dir_path, col_len=col_len_sagitto, is_saggitto=True)
df_saggito['x'] = df_saggito.iloc[:, 0:col_len_sagitto-1].values.tolist()

In [ ]:
df = pd.merge(df_neospectra[['x', 'Label']], df_saggito[['x', 'Label']], left_index=True, right_index=True)
df.shape

In [ ]:
df[(df.Label_x != df.Label_y)]

In [ ]:
df_saggito.loc['CO_AC_Y_5']